In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib

font = {'family' : 'normal',
#         'weight' : 'bold',
        'size'   : 18}
matplotlib.rc('font', **font)

### Import and normalize Fashion MNIST

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

### Train and evaluate neural networks, 10 repetitions

In [ ]:
import math
import scipy.stats

a1 = 1.0
a2 = 0.0
b1 = 1.0
b2 = 10.0
c = 0.5
d= 1.0

def sigmoid(x):
        return d*1.0/(1.0+tf.math.exp(-a1*(x-a2))) 

def ad_sigmoid(x):
        return d*1.0/(1.0+tf.math.exp(-a1*(x-a2))) *  (1.0/(1.0+tf.math.exp(b1*(x-b2)))*(1.0-c) + c )

    def scheduler(epoch, lr):
#     if epoch < 2:
#         return lr
#     else:
        return lr * np.exp(-0.1)

vaccs_all3 = []
vaccs_aves3 = []
vaccs_errs3 =[]
for pp in [0.1*i for i in range(11)]:
    accs=[]
    vaccs=[]
    
    nreps=0
    
    while nreps < 10:

        Kl = 30
        inputs = keras.Input(shape = (28,28))
        x = layers.Flatten()(inputs)
        x1 = layers.Dense(1000-int(1000*pp),activation=sigmoid)(x)
        x2 = layers.Dense(int(1000*pp),activation=ad_sigmoid)(x)
        x = layers.concatenate([x1,x2])
        x1 = layers.Dense(500-int(500*pp),activation=sigmoid)(x)
        x2 = layers.Dense(int(500*pp),activation=ad_sigmoid)(x)
        x = layers.concatenate([x1,x2])
        x1 = layers.Dense(250-int(250*pp),activation=sigmoid)(x)
        x2 = layers.Dense(int(250*pp),activation=ad_sigmoid)(x)
        x = layers.concatenate([x1,x2])
        x = layers.Dense(Kl, activation=None)(x)
        x1 = layers.Dense(250-int(250*pp),activation=sigmoid)(x)
        x2 = layers.Dense(int(250*pp),activation=ad_sigmoid)(x)
        x = layers.concatenate([x1,x2])
        x1 = layers.Dense(500-int(500*pp),activation=sigmoid)(x)
        x2 = layers.Dense(int(500*pp),activation=ad_sigmoid)(x)
        x = layers.concatenate([x1,x2])
        x1 = layers.Dense(1000-int(1000*pp),activation=sigmoid)(x)
        x2 = layers.Dense(int(1000*pp),activation=ad_sigmoid)(x)
        x = layers.concatenate([x1,x2])
        x = layers.Dense(28*28, activation=sigmoid)(x)
        outputs = layers.Reshape((28,28))(x)

        model = keras.Model(inputs = inputs, outputs = outputs)
#         model.summary()

        callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

        model.compile(optimizer='adam',
                      loss='MSE',
                      callbacks=[callback])

        history = model.fit(train_images, train_images, validation_data = [test_images,test_images], epochs=5)

        if math.isnan(history.history['val_loss'][-1]):
            continue
        
        else:
            nreps += 1
            accs.append(history.history['loss'])
            vaccs.append(history.history['val_loss'][-1])

    vaccs_all3.append(vaccs)
    vaccs_aves3.append(np.mean(vaccs))
    vaccs_errs3.append(scipy.stats.sem(vaccs))

### Plot accuracy vs proportion of type 1/2 neurons

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,5))
ax.scatter([10*i for i in range(11)], vaccs_aves3, c='black')
ax.errorbar([10*i for i in range(11)], vaccs_aves3, yerr=vaccs_errs3, capsize=3, c='black')

ax.set_xlabel("Proportion of type 2 neurons")
ax.set_xticks([0,50,100])
ax.set_xticklabels(['0','50','100%'])

axT = ax.twiny()
axT.set_xticks([0,0.5,1])
axT.set_xticklabels(['100','50','0%'])
axT.set_xlabel("Proportion of type 1 neurons")
axT.xaxis.labelpad = 10

ax.set_ylabel("MSE")

plt.subplots_adjust(left=0.3,bottom=0.3,top=0.7,right=0.9)
ax.tick_params(which='both', direction='in')

plt.savefig('Fig3B.eps')
